In [1]:
import pandas as pd
import torch
import numpy as np
from torch import nn, optim
import torch.nn.functional as F
import scipy.io
import torch.utils.data as Data
import time
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [2]:
import model 

In [3]:
device = 'cpu'

## Read Cromatine effects table
out_table = pd.read_csv('croamtine_reatures.csv',encoding = "ISO-8859-1")

In [4]:
deep_sea = model.deep_sea_nn()
deep_sea.load_state_dict(torch.load('deep_sea_model_params.pth', map_location='cpu'))
#deep_sea.to(device)
loss_func = nn.BCEWithLogitsLoss()
print(deep_sea)

deep_sea_nn(
  (conv1): Conv1d(4, 320, kernel_size=(8,), stride=(1,))
  (conv2): Conv1d(320, 480, kernel_size=(8,), stride=(1,))
  (conv3): Conv1d(480, 960, kernel_size=(8,), stride=(1,))
  (maxpool): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (drop1): Dropout(p=0.2)
  (drop2): Dropout(p=0.5)
  (linear1): Linear(in_features=50880, out_features=925, bias=True)
  (linear2): Linear(in_features=925, out_features=919, bias=True)
)


### TESTING SEQUENCE 

In [5]:
test_seq = 'TATCTCTCATGTTTCTGGTATAGATGGTATATATGTTAATCTTGTTCCTGAGGTCTGTTTTTTATTTTTGTCATTAAAGTGGGAATTAAATAGTTTTGTAGTGCATATAAATTAAAGAAAAAGTTCACATAAGCATATTTGCCAATCATCTCAAAATGCTATATTCTCCTTCACGGTTTTGAAAATAATTCAGGGTTTTCTCTTCCTCATTGCTTTCCCACCAACTGACAGTATTATTTTCTTAGTCATTTTACTGACCTTTGAAATTACTCCTTTGAGGTCTTCTAAAAAATTTTATGGGCTCTGCTGCTTTTTGGTGGCCTCCTTGTATCATTTATTCTATTACAGGACGACTTACAAAAGGAAGCACATAAATTGACCCATATACATATCCTATCATTGGGGAGTTTCTGTGCAAATGTTATTTATTGGAAGCTATTACTAAGAATTGTAAGAAAAATAATTGGTATTGATGCAGCTAGTATGGTTCCTGTAATTATCGTACTCAGCCACGTAAATCATAGCTATATGTAGCCAAAGATCCATGAACAAAATTTCCAGTAACATCATTATAATTCAAAAGGCAGACTTTCAGAACCAGACAGACTTGAATTTAAATTCTAGCTTTACCACACATGAATTTAACCTTGTGGAAGGTTAACCTATCTAAACTCATGTTTCTTCATTGGTAGCTGATAAAATTAAGGATCATGTATATAACCACCTAGTAGAGTTGTTTAAGAAACTGTTAGAATTCCATAAATTGTTAGTATTAATGAGTTTTTGTTGGACATGTGTTAGGCTAGGCCACTCCTTGACCTTCATAGAGGTATGGATTATGACACAAATTCTAAACTGTAGGTAGGCATGGCTTTGTAGCAAGTATTAAAATAGTAAATATTTTATTTTTATAAGATAAATGTAAACCTTTTAAAAGTTTCATTACATTTGTATTTATGAAATATCATCCTATATCAACTATAGAGAGAAGATCGCAAGA'
test_seq

'TATCTCTCATGTTTCTGGTATAGATGGTATATATGTTAATCTTGTTCCTGAGGTCTGTTTTTTATTTTTGTCATTAAAGTGGGAATTAAATAGTTTTGTAGTGCATATAAATTAAAGAAAAAGTTCACATAAGCATATTTGCCAATCATCTCAAAATGCTATATTCTCCTTCACGGTTTTGAAAATAATTCAGGGTTTTCTCTTCCTCATTGCTTTCCCACCAACTGACAGTATTATTTTCTTAGTCATTTTACTGACCTTTGAAATTACTCCTTTGAGGTCTTCTAAAAAATTTTATGGGCTCTGCTGCTTTTTGGTGGCCTCCTTGTATCATTTATTCTATTACAGGACGACTTACAAAAGGAAGCACATAAATTGACCCATATACATATCCTATCATTGGGGAGTTTCTGTGCAAATGTTATTTATTGGAAGCTATTACTAAGAATTGTAAGAAAAATAATTGGTATTGATGCAGCTAGTATGGTTCCTGTAATTATCGTACTCAGCCACGTAAATCATAGCTATATGTAGCCAAAGATCCATGAACAAAATTTCCAGTAACATCATTATAATTCAAAAGGCAGACTTTCAGAACCAGACAGACTTGAATTTAAATTCTAGCTTTACCACACATGAATTTAACCTTGTGGAAGGTTAACCTATCTAAACTCATGTTTCTTCATTGGTAGCTGATAAAATTAAGGATCATGTATATAACCACCTAGTAGAGTTGTTTAAGAAACTGTTAGAATTCCATAAATTGTTAGTATTAATGAGTTTTTGTTGGACATGTGTTAGGCTAGGCCACTCCTTGACCTTCATAGAGGTATGGATTATGACACAAATTCTAAACTGTAGGTAGGCATGGCTTTGTAGCAAGTATTAAAATAGTAAATATTTTATTTTTATAAGATAAATGTAAACCTTTTAAAAGTTTCATTACATTTGTATTTATGAAATATCATCCTATATCAACTATAGAGAGAAGATCGCAAG

In [6]:
len(test_seq)

1000

### One-hot encode sequence

In [7]:
ds_x_input = model.one_hot_encode_seq(test_seq)
ds_x_input.shape

(4, 1000)

In [8]:
ds_x_input

array([[0., 1., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.]])

### Deep Sea prediction

In [11]:
start_time = time.time()

test_out = deep_sea.foward(torch.FloatTensor(ds_x_input).view(1, 4, 1000))
            
test_output = torch.sigmoid(test_out.cpu().data).view(919).numpy()

model.get_title()
time_elapsed = time.time() - start_time
print('\n#### Predicction time elapsed: {} sec.'.format(time_elapsed))


    88888                     888888            
    8    8 eeee eeee eeeee    8      eeee eeeee 
    8e   8 8    8    8   8    8eeeee 8    8   8 
    88   8 8eee 8eee 8eee8        88 8eee 8eee8 
    88   8 88   88   88       e   88 88   88  8 
    88eee8 88ee 88ee 88       8eee88 88ee 88  8 
    

#### Predicction time elapsed: 0.02448892593383789 sec.


In [12]:
out_table['Deep SEA score'] = test_output

# Prediction

In [13]:
display(HTML(out_table.to_html()))

,Cromatine Feature,Cell Type,Deep SEA score
0,CTCF,Dnd41,0.000619
1,EZH2,Dnd41,0.006918
2,CTCF,GM12878,0.000995
3,EZH2,GM12878,0.000478
4,CHD1,H1-hESC,0.005998
5,CTCF,H1-hESC,0.005502
6,EZH2,H1-hESC,0.001181
7,JARID1A,H1-hESC,0.000453
8,RBBP5,H1-hESC,0.000341
9,CTCF,HeLa-S3,0.001547
